In [4]:
# from helper_functions.tourney_sim import simulate_tournament
#in v02 im just going to test 8 combo
import os
import numpy as np
import pickle
from tqdm import tqdm
from itertools import combinations
import pandas as pd
pd.set_option('display.max_columns', 90)

In [5]:
def implied_prob(line):
    if line < 0:
        ip = (-1*line)/(-1*line+100)
    else:
        ip=100/(line+100)
    return ip

In [6]:
def reverse_prob(p):
    if p <= .5:
        line = (100/p)-100
    else:
        line = (100*p)/(p-1)
    return line


In [7]:
def payout(line, bet):
    winnings=0
    if line < 0:
        winnings = bet/(-line/100)
    else:
        winnings = bet * (line/100)
    return (bet + winnings)

In [8]:
def combine_lines(lines=[]):
    probs=[]
    for l in lines:
        probs.append(implied_prob(l))
    total_prob=1
    for p in probs:
        total_prob*=p
    return reverse_prob(total_prob)


In [9]:
def find_ones_indexes(binary_tuple):
    indexes = []
    for index, bit in enumerate(binary_tuple):
        if bit == 1:
            indexes.append(index+1)
    return indexes

In [10]:
lines1 = {'Arizona':-310,
         'UConn':-800,
         'North Carolina':-198,
         'Iowa St.':-122,
         'Marqette':-285,
         'Purdue':-230,
         'Houston':-192,
         'Tennessee':-148}

In [11]:
lines2 = {'Clemson':250,
         'San Diego St.':550,
         'Alabama':164,
         'Illinois':102,
         'NC State':230,
         'Gonzaga':190,
         'Duke':160,
         'Creighton':124}

In [24]:
success_dict

{'Clemson': 0,
 'San Diego St.': 0,
 'Alabama': 1,
 'Illinois': 1,
 'NC State': 0,
 'Gonzaga': 0,
 'Duke': 0,
 'Creighton': 0}

In [12]:
probs1={'Arizona':.575,
       'UConn':.779,
       'North Carolina':.818,
       'Iowa St.':.723,
       'Marqette':.832,
       'Purdue':.566,
       'Houston':.416,
       'Tennessee':.481}

In [13]:
probs2={'Clemson':.425,
       'San Diego St.':.221,
       'Alabama':.182,
       'Illinois':.277,
       'NC State':.168,
       'Gonzaga':.434,
       'Duke':.584,
       'Creighton':.519}

In [14]:
lines_comb = dict(zip(list(lines1.keys()) + list(lines2.keys()),
                      list(lines1.values())+list(lines2.values())))

probs_comb = dict(zip(list(probs1.keys()) + list(probs2.keys()),
                      list(probs1.values())+list(probs2.values())))

In [15]:
lines_df = pd.DataFrame(zip(lines1,lines2))

In [16]:
lines_df

,0,1
0,Arizona,Clemson
1,UConn,San Diego St.
2,North Carolina,Alabama
3,Iowa St.,Illinois
4,Marqette,NC State
5,Purdue,Gonzaga
6,Houston,Duke
7,Tennessee,Creighton


In [17]:
n=8
binary_tuples = [tuple(int(bit) for bit in bin(i)[2:].zfill(n)) for i in range(2**n)]

In [18]:
rr_possibilities = binary_tuples.copy()

In [21]:
outer_df = pd.DataFrame()
for bets in tqdm(binary_tuples):
    picks = []
    for p in range(len(bets)):
        picks.append(lines_df[bets[p]][p])

        line_val = [lines_comb[x] for x in picks]
        lines=dict(zip(picks,line_val))

        prob_val = [probs_comb[x] for x in picks]
        probs=dict(zip(picks,prob_val))

    a1,b1,c1,d1,e1,f1,g1,h1 = picks


    best_rr = []
    moneys = []
    all_df = pd.DataFrame()

    for i in range(100):
        current_success = (np.random.choice([1,0],p=[probs[a1], 1-probs[a1]]),
                        np.random.choice([1,0],p=[probs[b1], 1-probs[b1]]),
                        np.random.choice([1,0],p=[probs[c1], 1-probs[c1]]),
                        np.random.choice([1,0],p=[probs[d1], 1-probs[d1]]),
                        np.random.choice([1,0],p=[probs[e1], 1-probs[e1]]),
                        np.random.choice([1,0],p=[probs[f1], 1-probs[f1]]),
                        np.random.choice([1,0],p=[probs[g1], 1-probs[g1]]),
                        np.random.choice([1,0],p=[probs[h1], 1-probs[h1]]))
        success_dict = dict(zip(lines.keys(), current_success))
        rr_moneys = []
        for c in rr_possibilities:
            cost = 0
            current_payout = 0
            current_rrs = find_ones_indexes(c)
            for i in current_rrs:
                current_combinations =list(combinations(lines.keys(),i))
                for combo in current_combinations:
                    line = combine_lines([lines[x] for x in combo])
                    cost += 0.1
                    win = True
                    for choice in combo:
                        if success_dict[choice] == 0:
                            win=False
                    if win:
                        current_payout += payout(line,0.1)
            rr_moneys.append(current_payout-cost)
        df = pd.DataFrame(zip(rr_possibilities,rr_moneys)).sort_values(by=1,ascending=False)
        df['iter']=i
        all_df=pd.concat([all_df, df])
        best_rr.append(df[0].iloc[0])
        moneys.append(df[1].iloc[0])
    new_df = all_df.groupby(0)[1].mean().reset_index().sort_values(by=1).copy()
    new_df.columns=['RR Set','Expected Return']
    new_df['Bet Combo'] = str(bets)
    outer_df = pd.concat([outer_df,new_df])

100%|██████████| 256/256 [29:21<00:00,  6.88s/it]


In [22]:
outer_df.sort_values(by='Expected Return')

,RR Set,Expected Return,Bet Combo
255,"(1, 1, 1, 1, 1, 1, 1, 1)",-21.650869,"(0, 1, 1, 1, 1, 0, 0, 0)"
254,"(1, 1, 1, 1, 1, 1, 1, 0)",-21.550869,"(0, 1, 1, 1, 1, 0, 0, 0)"
127,"(0, 1, 1, 1, 1, 1, 1, 1)",-21.389176,"(0, 1, 1, 1, 1, 0, 0, 0)"
126,"(0, 1, 1, 1, 1, 1, 1, 0)",-21.289176,"(0, 1, 1, 1, 1, 0, 0, 0)"
253,"(1, 1, 1, 1, 1, 1, 0, 1)",-20.850869,"(0, 1, 1, 1, 1, 0, 0, 0)"
...,...,...,...
191,"(1, 0, 1, 1, 1, 1, 1, 1)",78.787114,"(1, 1, 0, 0, 0, 1, 1, 1)"
126,"(0, 1, 1, 1, 1, 1, 1, 0)",79.408786,"(1, 1, 0, 0, 0, 1, 1, 1)"
254,"(1, 1, 1, 1, 1, 1, 1, 0)",79.665496,"(1, 1, 0, 0, 0, 1, 1, 1)"
127,"(0, 1, 1, 1, 1, 1, 1, 1)",80.730332,"(1, 1, 0, 0, 0, 1, 1, 1)"


In [ ]:
"""
interpretation - "Bet Combo" refers to picking favorites vs upsets - (1,1,1,1)
 means all upsets

 RR Set means which combos to take. (1,0,0,1) means take moneylines and 4-leg parlay
 (0,1,1,0) means take 2's and 3's

"""

In [23]:
best_rr = []
moneys = []
all_df = pd.DataFrame()

for i in tqdm(range(10000)):
    current_success = (np.random.choice([1,0],p=[probs[a1], 1-probs[a1]]),
                    np.random.choice([1,0],p=[probs[b1], 1-probs[b1]]),
                    np.random.choice([1,0],p=[probs[c1], 1-probs[c1]]),
                    np.random.choice([1,0],p=[probs[d1], 1-probs[d1]]),
                    np.random.choice([1,0],p=[probs[e1], 1-probs[e1]]),
                    np.random.choice([1,0],p=[probs[f1], 1-probs[f1]]),
                    np.random.choice([1,0],p=[probs[g1], 1-probs[g1]]),
                    np.random.choice([1,0],p=[probs[h1], 1-probs[h1]]))
    success_dict = dict(zip(lines.keys(), current_success))
    rr_moneys = []
    for c in rr_possibilities:
        cost = 0
        current_payout = 0
        current_rrs = find_ones_indexes(c)
        for i in current_rrs:
            current_combinations =list(combinations(lines.keys(),i))
            for combo in current_combinations:
                line = combine_lines([lines[x] for x in combo])
                cost += 0.1
                win = True
                for choice in combo:
                    if success_dict[choice] == 0:
                        win=False
                if win:
                    current_payout += payout(line,0.1)
        rr_moneys.append(current_payout-cost)
    df = pd.DataFrame(zip(rr_possibilities,rr_moneys)).sort_values(by=1,ascending=False)
    df['iter']=i
    all_df=pd.concat([all_df, df])
    best_rr.append(df[0].iloc[0])
    moneys.append(df[1].iloc[0])


100%|██████████| 10000/10000 [10:23<00:00, 16.03it/s]


In [46]:
new_df = all_df.groupby(0)[1].mean().reset_index().sort_values(by=1).copy()
new_df.columns=['RR Set','Expected Return']

In [47]:
new_df['Expected Return'] *= 10

In [48]:
new_df

,RR Set,Expected Return
0,"(0, 0, 0, 0)",0.000000
1,"(0, 0, 0, 1)",0.143426
8,"(1, 0, 0, 0)",0.212492
9,"(1, 0, 0, 1)",0.355919
2,"(0, 0, 1, 0)",0.507477
4,"(0, 1, 0, 0)",0.576267
3,"(0, 0, 1, 1)",0.650904
5,"(0, 1, 0, 1)",0.719694
10,"(1, 0, 1, 0)",0.719970
12,"(1, 1, 0, 0)",0.788760


In [136]:
all_df.groupby(0)[1].mean().reset_index().sort_values(by=1)

,0,1
0,"(0, 0, 0, 0)",0.000000
1,"(0, 0, 0, 1)",0.071078
8,"(1, 0, 0, 0)",0.075383
9,"(1, 0, 0, 1)",0.146461
2,"(0, 0, 1, 0)",0.211975
4,"(0, 1, 0, 0)",0.222267
3,"(0, 0, 1, 1)",0.283053
10,"(1, 0, 1, 0)",0.287357
5,"(0, 1, 0, 1)",0.293345
12,"(1, 1, 0, 0)",0.297649
